# 第1步：导入库和加载图像

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Noto Sans CJK SC', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 加载图像
img = cv2.imread("table_sample.jpg")  # 替换为你的图片路径
if img is None:
    raise ValueError("无法加载图像，请检查文件路径。")

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(8,6),dpi=300)
plt.imshow(img_rgb)
plt.title('原始拼豆图纸')
plt.axis('off')
plt.show()

print(f"图像尺寸: {img.shape}")

# 第2步：多通道梯度计算

In [ ]:
# Sobel 算子 检测水平和竖直的边缘
def get_gradient_sobel(channel:cv2.Mat,ksize:int) -> np.ndarray:
    grad_x = cv2.Sobel(channel, cv2.CV_64F, 1, 0, ksize=ksize)
    grad_y = cv2.Sobel(channel, cv2.CV_64F, 0, 1, ksize=ksize)
    return np.sqrt(grad_x**2 + grad_y**2)

# Scharr 算子
def get_gradient_scharr(channel: cv2.Mat) -> np.ndarray:
    grad_x = cv2.Scharr(channel, cv2.CV_64F, 1, 0)
    grad_y = cv2.Scharr(channel, cv2.CV_64F, 0, 1)
    return np.sqrt(grad_x**2 + grad_y**2)

# Laplacian 算子
def get_gradient_laplacian(channel: cv2.Mat,ksize: int) -> np.ndarray:
    laplacian = cv2.Laplacian(channel, cv2.CV_64F,ksize=ksize)
    return np.abs(laplacian)

def get_gradient(channel) -> np.ndarray:
    return get_gradient_scharr(channel=channel)  # 使用Canny边缘检测

# RGB三通道分别计算
b, g, r = cv2.split(img)
grad_r = get_gradient(r)
grad_g = get_gradient(g)
grad_b = get_gradient(b)

# LAB色差通道
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l, a, b_lab = cv2.split(lab)
grad_a = get_gradient(a)  # 红绿色差
grad_b_lab = get_gradient(b_lab)  # 黄蓝色差

# 可视化各通道梯度
fig, axes = plt.subplots(2, 3, figsize=(15, 8), dpi=300)
axes[0,0].imshow(grad_r, cmap='hot'), axes[0,0].set_title('R通道梯度'), axes[0,0].axis('off')
axes[0,1].imshow(grad_g, cmap='hot'), axes[0,1].set_title('G通道梯度'), axes[0,1].axis('off')
axes[0,2].imshow(grad_b, cmap='hot'), axes[0,2].set_title('B通道梯度'), axes[0,2].axis('off')
axes[1,0].imshow(grad_a, cmap='hot'), axes[1,0].set_title('A通道梯度(红绿色差)'), axes[1,0].axis('off')
axes[1,1].imshow(grad_b_lab, cmap='hot'), axes[1,1].set_title('B通道梯度(黄蓝色差)'), axes[1,1].axis('off')
axes[1,2].axis('off')
plt.tight_layout()
plt.show()

# 第3步：智能权重融合

In [ ]:
# 计算每个通道的平均强度作为权重
weights = {
    'r': np.mean(grad_r),
    'g': np.mean(grad_g),
    'b': np.mean(grad_b),
    'a': np.mean(grad_a),
    'b_lab': np.mean(grad_b_lab)
}

print("各通道平均梯度强度:")
for k, v in weights.items():
    print(f"{k}: {v:.2f}")

# 归一化并融合
total_weight = sum(weights.values())
fused_gradient = (
    (weights['r']/total_weight) * grad_r +
    (weights['g']/total_weight) * grad_g +
    (weights['b']/total_weight) * grad_b

    # + (weights['a']/total_weight) * grad_a * 1.5 +    # 色差通道加权
    # (weights['b_lab']/total_weight) * grad_b_lab * 1.5
)

# 对融合图像进行高斯滤波
# fused_gradient = cv2.GaussianBlur(fused_gradient.astype(np.float32), (5, 5), 0)
# fused_gradient = cv2.bilateralFilter(fused_gradient.astype(np.float32), d=9, sigmaColor=75, sigmaSpace=75)

# 使用60%阈值的二值图进行线检测
threshold_55 = fused_gradient > np.percentile(fused_gradient, 60)
binary_55 = (threshold_55 * 255).astype(np.uint8)
binary_55 = cv2.GaussianBlur(binary_55.astype(np.float32), (5, 5), 0)
binary_55 = binary_55 > np.percentile(binary_55, 60)
binary_55 = (binary_55 * 255).astype(np.uint8)

plt.figure(figsize=(12, 4),dpi=300)
plt.subplot(131), plt.imshow(grad_r, cmap='hot'), plt.title('单通道(R)'), plt.axis('off')
plt.subplot(132), plt.imshow(fused_gradient, cmap='hot'), plt.title('多通道融合'), plt.axis('off')
plt.subplot(133), plt.imshow(binary_55, cmap='gray'), plt.title('60%阈值预览'), plt.axis('off')
plt.show()

## 生成文字掩膜（单词分布概率图）

### 导入需要的库并环境检测

In [ ]:
import re
# === 环境检测：CUDA 与 ONNXRuntime ===
print("=== 环境检测 ===")
# 检查 onnxruntime GPU provider
try:
    import onnxruntime as ort
    print("ONNXRuntime 可用提供者:", ort.get_available_providers())
except ImportError:
    print("未安装 onnxruntime")
print("================")

import onnxruntime as ort

def inspect_onnx_model(onnx_path:str,use_gpu:bool) -> ort.InferenceSession | None:
    """检查 ONNX 模型的输入输出信息"""
    try:
        providers = ["CUDAExecutionProvider", "CPUExecutionProvider"] if use_gpu else ["CPUExecutionProvider"]
        if use_gpu is True:
            print("使用 GPU 加速，提供者:", providers)
        else:
            print("使用 CPU 运行，提供者:", providers)
        session = ort.InferenceSession(onnx_path, providers=providers)

        print(f"\n=== 模型信息：{onnx_path} ===")

        # 输入信息
        for i, input_info in enumerate(session.get_inputs()):
            print(f"输入 {i}:")
            print(f"  名称: {input_info.name}")
            print(f"  形状: {input_info.shape}")
            print(f"  类型: {input_info.type}")

            # 检查是否有动态维度
            if any(isinstance(dim, str) or dim == -1 for dim in input_info.shape):
                print(f"  ✓ 支持动态尺寸")
            else:
                print(f"  ⚠️ 固定尺寸，必须严格匹配")

        # 输出信息
        for i, output_info in enumerate(session.get_outputs()):
            print(f"输出 {i}:")
            print(f"  名称: {output_info.name}")
            print(f"  形状: {output_info.shape}")
            print(f"  类型: {output_info.type}")

        print("=" * 50)
        return session

    except Exception as e:
        print(f"⚠️ 无法检查模型信息: {e}")
        return None

# 文件路径
onnx_path = "Text-Remove-Model/ch_PP-OCRv5_server_det.onnx"
# 图片路径
img_path = "table_sample.jpg"
# 使用GPU
use_gpu = True

# 首先检查模型的输入要求
session = inspect_onnx_model(onnx_path,use_gpu)

### 自动处理图片分辨率和模型对齐

In [ ]:
# 载入图片
img = cv2.imread(img_path)
if img is None:
    print(f"无法读取图像，{img_path}")

# 获取模型输入形状
model_input_shape = session.get_inputs()[0].shape

# 智能推断输入尺寸
target_size = "auto"  # 默认自动

# 最优先：从模型本身获取固定尺寸
if model_input_shape and len(model_input_shape) >= 4:
    # 检查是否是固定尺寸 (非动态维度)
    h_dim, w_dim = model_input_shape[2], model_input_shape[3]
    if isinstance(h_dim, int) and isinstance(w_dim, int) and h_dim == w_dim:
        target_size = h_dim
        print(f"从模型获取固定输入尺寸: {target_size}x{target_size}")

# 次优先：从文件名推断
if target_size == "auto":
    # 使用正则表达式从文件名中提取尺寸信息
    size_pattern = r'(\d+)x\1'  # 匹配如 1024x1024, 2400x2400 等格式
    size_match = re.search(size_pattern, onnx_path)

    if size_match:
        target_size = int(size_match.group(1))
        print(f"从文件名检测到固定尺寸: {target_size}x{target_size}")
    else:
        # 如果没有找到标准格式，尝试匹配 NxN 格式（如 600x600, 800x800 等）
        general_size_pattern = r'(\d+)x(\d+)'  # 匹配任意 NxM 格式
        general_match = re.search(general_size_pattern, onnx_path)
        if general_match:
            width, height = int(general_match.group(1)), int(general_match.group(2))
            if width == height:  # 只接受正方形输入
                target_size = width
                print(f"从文件名推断尺寸: {target_size}x{target_size}")
            else:
                print(f"从文件名检测到非正方形尺寸 {width}x{height}，将使用 {max(width, height)}x{max(width, height)}")
                target_size = max(width, height)

    # 对于从文件名中获取的尺寸，也确保是32的倍数
    if isinstance(target_size, int):
        original_size = target_size
        target_size = ((target_size + 31) // 32) * 32
        if original_size != target_size:
            print(f"输入尺寸从 {original_size} 调整为 {target_size} (32的倍数)")

# 如果没有从文件名中找到尺寸，读取图像来自动判断到最近的一个32倍
if target_size == "auto":
    temp_img = img
    if temp_img is not None:
        h, w = temp_img.shape[:2]
        max_dim = max(h, w)

        # 最大限制 4096
        target_size = min(4096,max_dim)

        # 确保是32的倍数
        target_size = ((target_size + 31) // 32) * 32
        print(f"图像尺寸: {w}x{h}, 自动选择模型输入尺寸: {target_size}x{target_size}")
    else:
        target_size = 1024  # 默认值

print(f"加载模型: {onnx_path}")
print(f"处理图像: {img_path}")
print(f"使用输入尺寸: {target_size}x{target_size}")
# print(f"推理引擎: {'ONNXRuntime' if use_ort else 'OpenCV DNN'}")
print(f"GPU加速: {'开启' if use_gpu else '关闭'}")

### 图像预处理

In [ ]:
def preprocess_image(img: np.ndarray,target_size:int = 1024) -> tuple[np.ndarray, float, tuple[int, int]]:
    """预处理方式 - 保持原始比例并缩放到对应的大小画布上"""
    h, w = img.shape[:2]

    # 计算缩放比例，保持长宽比
    scale = target_size / max(h, w)
    new_h, new_w = int(h * scale), int(w * scale)

    # resize图像
    img_resized = cv2.resize(img, (new_w, new_h))

    # 创建目标尺寸的画布（填充黑色）
    canvas = np.zeros((target_size, target_size, 3), dtype=np.uint8)

    # 将resize后的图像放到画布中央
    start_h = (target_size - new_h) // 2
    start_w = (target_size - new_w) // 2
    canvas[start_h:start_h+new_h, start_w:start_w+new_w] = img_resized

    return canvas, scale, (start_w, start_h)

### 将概率图缩放回原图函数

In [ ]:
def generate_original_prob_map(
        prob_map:np.ndarray,
        scale:float,
        offset:tuple[int,int],
        original_size:tuple[int,int],
        target_size:int
        ):
    """将概率图缩放回原图尺寸"""
    original_w, original_h = original_size
    start_w, start_h = offset

    # 计算原图在预处理图像中的有效区域尺寸
    scaled_w = int(original_w * scale)
    scaled_h = int(original_h * scale)

    # 从概率图中提取有效区域（去除padding）
    end_h = min(start_h + scaled_h, target_size)
    end_w = min(start_w + scaled_w, target_size)

    # 提取有效区域
    valid_prob = prob_map[start_h:end_h, start_w:end_w]

    # 缩放回原图尺寸
    if valid_prob.size > 0:
        prob_resized = cv2.resize(valid_prob, (original_w, original_h))
        # 归一化到0-255用于保存
        prob_normalized = (prob_resized * 255).astype(np.uint8)
        return prob_normalized
    else:
        return np.zeros((original_h, original_w), dtype=np.uint8)


### 文本检测

In [ ]:
def detect_word(
        onnx_session:ort.InferenceSession,
        img:np.ndarray,
        target_size:int=1024,
        threshold:float=0.3,
        use_gpu:bool=False):
    "文件检测函数"
    # 获取原始图像尺寸
    original_h, original_w = img.shape[:2]
    print(f"原始图像尺寸: {original_w}x{original_h}")

    # 预处理
    processed_img, scale, offset = preprocess_image(img, target_size)
    print(f"预处理后尺寸: {processed_img.shape}")
    print(f"缩放比例: {scale:.4f}, 偏移: {offset}")

    # 检查模型要求的通道数
    model_input_shape = onnx_session.get_inputs()[0].shape
    if len(model_input_shape) >= 2 and isinstance(model_input_shape[1], int):
        expected_channels = model_input_shape[1]
        print(f"模型期望输入通道数: {expected_channels}")
    else:
        expected_channels = 3  # 默认RGB三通道

    # 创建blob
    if expected_channels == 1:
        # 单通道模型：转换为灰度图
        gray_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
        blob = cv2.dnn.blobFromImage(gray_img,
                                    scalefactor=1.0/255.0,
                                    size=(target_size, target_size),
                                    mean=(0.485,),  # 单通道均值
                                    swapRB=False,
                                    crop=False)
        print(f"单通道模型，使用灰度图输入")
    else:
        # 多通道模型：RGB
        blob = cv2.dnn.blobFromImage(processed_img,
                                    scalefactor=1.0/255.0,
                                    size=(target_size, target_size),
                                    mean=(0.485, 0.456, 0.406),  # ImageNet均值
                                    swapRB=True,  # BGR->RGB
                                    crop=False)
        print(f"多通道模型，使用RGB输入")

    print(f"输入 blob 形状: {blob.shape}")

    # 使用 ONNXRuntime 进行推理
    print(f"使用 ONNXRuntime 进行推理...")
    try:
        input_name = onnx_session.get_inputs()[0].name
        outputs = onnx_session.run(None, {input_name: blob.astype(np.float32)})
        print(f"推理成功")
    except Exception as e:
        print(f"推理失败: {e}")
        return None

    print(f"模型输出数量: {len(outputs)}")
    for i, output in enumerate(outputs):
        print(f"输出 {i} 形状: {output.shape}, 数值范围: [{np.min(output):.4f}, {np.max(output):.4f}]")

    # 处理输出（通常是概率图）
    if len(outputs) == 0:
        return None

    # 取第一个输出作为概率图
    prob_map = outputs[0]

    # 处理不同的输出格式
    if len(prob_map.shape) == 4:  # [N, C, H, W]
        prob_map = prob_map[0, 0]  # 取第一个batch和channel
    elif len(prob_map.shape) == 3:  # [N, H, W]
        prob_map = prob_map[0]

    print(f"概率图形状: {prob_map.shape}")

    # 调整到预处理图像尺寸
    if prob_map.shape != (target_size, target_size):
        prob_map = cv2.resize(prob_map, (target_size, target_size))

    # 正确生成原图尺寸的概率图
    prob_original = generate_original_prob_map(prob_map,scale,offset,(original_w,original_h),target_size)

    return prob_original

# 对推理得图像文字概率图
prob_map = detect_word(onnx_session=session, img=img, target_size=target_size, use_gpu=use_gpu)
# 根据概率图提取边缘并附加在原图上，使用绿色
if prob_map is not None:
    # 轻度平滑，降低噪声
    prob_blur = cv2.GaussianBlur(prob_map, (5, 5), 0)

    # 自适应Canny阈值（基于中位数）
    v = np.median(prob_blur)
    lower = int(max(0, 0.66 * v))
    upper = int(min(255, 1.33 * v))

    text_edges = cv2.Canny(prob_blur, lower, upper)

    # 可选：稍微膨胀让边缘更清晰
    kernel = np.ones((3, 3), np.uint8)
    text_edges = cv2.dilate(text_edges, kernel, iterations=1)

    # 叠加到原始RGB图像上（绿色）
    overlay_with_text_edges = img_rgb.copy()
    overlay_with_text_edges[text_edges > 0] = [0, 255, 0]

    # 可视化：概率图、边缘、叠加结果
    plt.figure(figsize=(16, 5), dpi=300)
    plt.subplot(131); plt.imshow(prob_map, cmap='gray'); plt.title('文字概率图'); plt.axis('off')
    plt.subplot(132); plt.imshow(text_edges, cmap='gray'); plt.title('概率边缘'); plt.axis('off')
    plt.subplot(133); plt.imshow(overlay_with_text_edges); plt.title('边缘叠加原图(绿色)'); plt.axis('off')
    plt.tight_layout(); plt.show()
else:
    print('未能生成概率图，跳过边缘叠加。')


### 将多通道融合的边缘图或者60%阈值图减去概率图消除文字的影响

In [ ]:
# 将文字概率图做二值化
if 'prob_map' not in globals() or prob_map is None:
    print('未找到概率图 prob_map，跳过本单元。')
else:
    # 1) 概率图二值化（Otsu）得到文字掩膜
    _, text_mask = cv2.threshold(prob_map, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # 形态学闭运算，填补小孔洞
    kernel = np.ones((3, 3), np.uint8)
    text_mask = cv2.morphologyEx(text_mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    # 2) 在融合的梯度图中将文字区域置零，得到“去文字”的融合多通道图
    fused_gradient_no_text = np.array(fused_gradient, dtype=np.float32).copy()
    fused_gradient_no_text[text_mask > 0] = 0.0

    # 3) 按照60分位重新生成新的阈值图（基于去文字后的融合图）
    p = 60
    thr = np.percentile(fused_gradient_no_text, p)
    binary_60_no_text = (fused_gradient_no_text > thr).astype(np.uint8) * 255
    # 与之前流程保持一致：平滑后再次按60分位阈值
    tmp = cv2.GaussianBlur(binary_60_no_text.astype(np.float32), (5, 5), 0)
    thr2 = np.percentile(tmp, p)
    binary_60_no_text = ((tmp > thr2).astype(np.uint8) * 255)

    # 4) 将之前的60%阈值图与原始文字概率图作差，得到“消除文字之后的非阈值图”
    if binary_55.shape != prob_map.shape:
        prob_map_resized = cv2.resize(prob_map, (binary_55.shape[1], binary_55.shape[0]))
    else:
        prob_map_resized = prob_map
    non_threshold_removed = cv2.subtract(binary_55, prob_map_resized)

    # 5) 可视化结果
    plt.figure(figsize=(20, 12), dpi=300)
    plt.subplot(2, 4, 1); plt.imshow(prob_map, cmap='gray'); plt.title('原始文字概率图'); plt.axis('off')
    plt.subplot(2, 4, 2); plt.imshow(text_mask, cmap='gray'); plt.title('文字概率二值掩膜'); plt.axis('off')
    plt.subplot(2, 4, 3); plt.imshow(fused_gradient, cmap='hot'); plt.title('原始多通道融合图'); plt.axis('off')
    plt.subplot(2, 4, 4); plt.imshow(fused_gradient_no_text, cmap='hot'); plt.title('去文字后的融合图'); plt.axis('off')
    plt.subplot(2, 4, 5); plt.imshow(binary_55, cmap='gray'); plt.title('原60%阈值图'); plt.axis('off')
    plt.subplot(2, 4, 6); plt.imshow(binary_60_no_text, cmap='gray'); plt.title('去文字后60%阈值图'); plt.axis('off')
    plt.subplot(2, 4, 7); plt.imshow(non_threshold_removed, cmap='gray'); plt.title('原60%阈值图 - 概率图'); plt.axis('off')
    # 第8幅：叠加文字掩膜可视化（可选）
    mask_overlay = img_rgb.copy()
    mask_overlay[text_mask > 0] = [255, 0, 255]
    plt.subplot(2, 4, 8); plt.imshow(mask_overlay); plt.title('文字掩膜叠加原图'); plt.axis('off')
    plt.tight_layout(); plt.show()

# 第4步：基于形态学的网格线和交叉点检测（最终版本）

In [ ]:
# 提取投影的上边缘数据
def extract_top_edge(projection_data, std_multiplier=2) -> tuple[np.ndarray, float, np.ndarray]:
    """
    提取投影数据的上边缘（顶部）数据点

    Args:
        projection_data: 一维投影数据数组
        std_multiplier: 标准差倍数，用于确定阈值

    Returns:
        top_edge_data: 超过阈值的数据点
        threshold: 使用的阈值
        indices: 超过阈值的数据点索引
    """
    # 计算均值和标准差
    mean = np.mean(projection_data)
    std = np.std(projection_data)

    # 定义阈值（均值 + n倍标准差）
    # threshold = mean + std_multiplier * std

    # 高分位点为阈值
    threshold = np.quantile(projection_data, 0.95)

    # 找到超过阈值的索引
    indices = np.where(projection_data >= threshold)[0]

    # 提取顶部数据
    top_edge_data = projection_data[indices]

    print(f"投影数据统计: 均值={mean:.2f}, 标准差={std:.2f}")
    print(f"阈值 (均值+{std_multiplier}σ): {threshold:.2f}")
    print(f"提取到 {len(top_edge_data)} 个顶部数据点")

    return top_edge_data, threshold, indices
def extract_grid_lines_morphology(binary_img, min_line_length=15,max_line_length=400, iterations=2):
    """
    使用形态学的自适应网格操作提取水平线和垂直线
    """
    print(f"输入图像尺寸: {binary_img.shape}")

    # 初步检测以估算网格密度
    initial_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 1))
    temp_horizontal = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, initial_kernel, iterations=2)
    # 投影每一行的白色像素（检测到的水平线像素）的总数
    temp_projection = np.sum(temp_horizontal, axis=1)

    from scipy.signal import find_peaks
    # 找到所有峰值
    temp_peaks, properties = find_peaks(temp_projection, height=np.max(temp_projection) * 0.1, distance=5)
    # 获取上边缘峰值数据
    filtered_indices = extract_top_edge(properties['peak_heights'],1)[2]
    # mean_peak_height = np.mean(properties['peak_heights'])
    # # 过滤掉低于平均值10%的峰值
    # filtered_indices = np.where(properties['peak_heights'] >= mean_peak_height * 0.90)[0]
    temp_peaks = temp_peaks[filtered_indices]
    # 计算最大的峰值和最小的峰值的差，获得网格的总长度
    if len(temp_peaks) > 0:
        grid_length = np.max(temp_peaks) - np.min(temp_peaks)
    else:
        grid_length = 0
    print(f"网格线总跨度: {grid_length} 像素")

    # 计算网格密度并自适应调整线段长度
    density = len(temp_peaks) / grid_length if len(temp_peaks) > 0 else 0.01
    da = grid_length / len(temp_peaks)
    adaptive_line_length = max(min_line_length, min(max_line_length, int(0.01/density * 100)))

    print(f"检测到网格密度: {density:.4f} 线/像素")
    print(f"自适应线段长度: {adaptive_line_length} 像素")

    # 创建结构元素
    # 水平线检测核 - 长水平线，窄垂直线
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (adaptive_line_length, 1))
    # 垂直线检测核 - 窄水平线，长垂直线
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, adaptive_line_length))

    # 形态学开运算提取线条
    horizontal_lines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN,
                                      horizontal_kernel, iterations=iterations)
    vertical_lines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN,
                                    vertical_kernel, iterations=iterations)

    print(f"提取的水平线像素数: {np.sum(horizontal_lines > 0)}")
    print(f"提取的垂直线像素数: {np.sum(vertical_lines > 0)}")

    return horizontal_lines, vertical_lines

def find_line_positions(line_mask, direction='horizontal'):
    """
    从线条掩码中提取线条的精确位置
    """
    if direction == 'horizontal':
        # 水平线：在y方向上投影
        projection = np.sum(line_mask, axis=1)
    else:
        # 垂直线：在x方向上投影
        projection = np.sum(line_mask, axis=0)

    # 找到投影的峰值位置
    from scipy.signal import find_peaks

    # 设置峰值检测参数
    height_threshold = np.max(projection) * 0.1  # 10%的最大值作为阈值
    min_distance = 5  # 最小间距

    peaks, properties = find_peaks(projection, height=height_threshold, distance=min_distance)
    if len(peaks) > 0:
        mean_peak_height = np.mean(properties['peak_heights'])
        filtered_indices = np.where(properties['peak_heights'] >= mean_peak_height * 0.90)[0]
        peaks = peaks[filtered_indices]

    return peaks, projection

def detect_grid_intersections_morphology(horizontal_lines, vertical_lines):
    """
    检测网格交叉点
    """
    # 找到交叉点：水平线和垂直线的交集
    intersections_mask = cv2.bitwise_and(horizontal_lines, vertical_lines)

    # 使用连通组件找到交叉点位置
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        intersections_mask, connectivity=8)

    # 过滤掉面积太小的组件
    min_area = 2  # 最小面积阈值
    valid_intersections = []

    for i in range(1, num_labels):  # 跳过背景标签0
        area = stats[i, cv2.CC_STAT_AREA]
        if area >= min_area:
            cx, cy = centroids[i]
            valid_intersections.append((cx, cy))

    print(f"检测到 {len(valid_intersections)} 个网格交叉点")

    return np.array(valid_intersections), intersections_mask

# 执行形态学网格检测
print("=== 开始形态学网格检测 ===")

# 1. 提取水平线和垂直线
horizontal_lines, vertical_lines = extract_grid_lines_morphology(
    binary_55, min_line_length=5,max_line_length=400, iterations=2)

# 2. 获取线条位置
horizontal_positions, h_projection = find_line_positions(horizontal_lines, 'horizontal')
vertical_positions, v_projection = find_line_positions(vertical_lines, 'vertical')

print(f"检测到 {len(horizontal_positions)} 条水平线")
print(f"检测到 {len(vertical_positions)} 条垂直线")

# 3. 检测交叉点
grid_intersections, intersection_mask = detect_grid_intersections_morphology(
    horizontal_lines, vertical_lines)

# 第5步：形态学检测结果可视化

In [ ]:
plt.figure(figsize=(20, 16), dpi=300)

# 1. 原始图像
plt.subplot(3, 4, 1)
plt.imshow(img_rgb)
plt.title('原始拼豆图纸')
plt.axis('off')

# 2. 60%阈值二值图
plt.subplot(3, 4, 2)
plt.imshow(binary_55, cmap='gray')
plt.title('60%阈值二值图')
plt.axis('off')

# 3. 提取的水平线
plt.subplot(3, 4, 3)
plt.imshow(horizontal_lines, cmap='gray')
plt.title(f'水平线提取\n{len(horizontal_positions)}条线')
plt.axis('off')

# 4. 提取的垂直线
plt.subplot(3, 4, 4)
plt.imshow(vertical_lines, cmap='gray')
plt.title(f'垂直线提取\n{len(vertical_positions)}条线')
plt.axis('off')

# 5. 水平线投影分析
plt.subplot(3, 4, 5)
plt.plot(h_projection, range(len(h_projection)), color='blue')
plt.scatter(h_projection[horizontal_positions], horizontal_positions,
           color='red', s=50, zorder=5)
plt.title('水平线投影')
plt.xlabel('投影强度')
plt.ylabel('Y坐标')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3)

# 6. 垂直线投影分析
plt.subplot(3, 4, 6)
plt.plot(v_projection, color='green')
plt.scatter(vertical_positions, v_projection[vertical_positions],
           color='red', s=50, zorder=5)
plt.title('垂直线投影')
plt.xlabel('X坐标')
plt.ylabel('投影强度')
plt.grid(True, alpha=0.3)

# 7. 交叉点掩码
plt.subplot(3, 4, 7)
plt.imshow(intersection_mask, cmap='gray')
plt.title('交叉点掩码')
plt.axis('off')

# 8. 合并的线条图
plt.subplot(3, 4, 8)
combined_lines = cv2.bitwise_or(horizontal_lines, vertical_lines)
plt.imshow(combined_lines, cmap='gray')
plt.title('合并线条')
plt.axis('off')

# 9. 线条叠加在原图上
plt.subplot(3, 4, 9)
lines_overlay = img_rgb.copy()
# 水平线 - 绿色
lines_overlay[horizontal_lines > 0] = [0, 255, 0]
# 垂直线 - 红色
lines_overlay[vertical_lines > 0] = [255, 0, 0]
# 交叉点 - 黄色
lines_overlay[intersection_mask > 0] = [255, 255, 0]
plt.imshow(lines_overlay)
plt.title('线条叠加原图\n红=垂直,绿=水平,黄=交叉')
plt.axis('off')

# 10. 交叉点标记
plt.subplot(3, 4, 10)
intersection_result = img_rgb.copy()
for i, (x, y) in enumerate(grid_intersections):
    cv2.circle(intersection_result, (int(x), int(y)), 4, (255, 0, 0), -1)
    # 每50个点标注序号
    if i % 50 == 0:
        cv2.putText(intersection_result, str(i), (int(x)+5, int(y)-5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
plt.imshow(intersection_result)
plt.title(f'网格交叉点\n共{len(grid_intersections)}个')
plt.axis('off')

# 11. 网格间距分析
plt.subplot(3, 4, 11)
if len(grid_intersections) > 4:
    from scipy.spatial.distance import cdist

    # 计算最近邻距离
    distances = cdist(grid_intersections, grid_intersections)
    np.fill_diagonal(distances, np.inf)
    min_distances = np.min(distances, axis=1)

    plt.hist(min_distances, bins=20, alpha=0.7, color='blue', edgecolor='black')

    # 统计信息
    median_spacing = np.median(min_distances)
    mean_spacing = np.mean(min_distances)
    std_spacing = np.std(min_distances)

    plt.axvline(median_spacing, color='red', linestyle='--',
               label=f'中位数: {median_spacing:.1f}')
    plt.axvline(mean_spacing, color='green', linestyle='--',
               label=f'平均值: {mean_spacing:.1f}')

    plt.title(f'网格间距分布\n标准差: {std_spacing:.1f}')
    plt.xlabel('距离(像素)')
    plt.ylabel('频次')
    plt.legend()
    plt.grid(True, alpha=0.3)
else:
    plt.text(0.5, 0.5, '交点数据不足', ha='center', va='center',
             transform=plt.gca().transAxes)
    plt.title('网格间距分析')

# 12. 交点分布图
plt.subplot(3, 4, 12)
if len(grid_intersections) > 0:
    plt.scatter(grid_intersections[:, 0], grid_intersections[:, 1],
               alpha=0.7, s=20, c='red')
    plt.title(f'交点分布图\n{len(grid_intersections)}个交点')
    plt.xlabel('X坐标')
    plt.ylabel('Y坐标')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3)

    # 显示网格的大概范围
    x_min, x_max = np.min(grid_intersections[:, 0]), np.max(grid_intersections[:, 0])
    y_min, y_max = np.min(grid_intersections[:, 1]), np.max(grid_intersections[:, 1])

    plt.text(0.02, 0.98, f'X范围: {x_min:.0f}-{x_max:.0f}\nY范围: {y_min:.0f}-{y_max:.0f}',
             transform=plt.gca().transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()